### Visualization of our EEG data 
A notebook with different methods to visualize our data and results of classification  
Author: Shateesh Bhugwansing

## NOTE (10/31): 
The sensor spacing is off. I need to figure out why.
things to try (for next week):
- plotting 2D position data

In [32]:
# %run ../preprocessing/StimCodes.ipynb
import mne
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

# makes pop up window for plots
%matplotlib qt 

import matplotlib
matplotlib.rcParams.update({'font.size': 11})

In [ ]:
# read in data 
epoch_path = '/Volumes/SB/EpochedEEG/20131216_1441_epo.fif'
epoch = mne.read_epochs(epoch_path, preload=True)

In [34]:
montage = mne.channels.read_montage(kind="ANT_DukeWaveGuard_128_electrode_montages_updated_V2")
# V2 is the version that's rotated to the right

In [35]:
from pylab import rcParams
rcParams['figure.figsize'] = 20, 15
montage.plot();

1 duplicate electrode labels found:
RA2 /RA4 
Plotting 124 unique labels.
Channel names are not unique, found duplicates for: {'RA5 ', 'RR10'}. Applying running numbers for duplicates.
The following EEG sensors did not have a position specified in the selected montage: ['RA5 -1', 'RR10-1']. Their position has been left untouched.


<ipython-input-35-8c0958edec32>:3: RuntimeWarning: Channel names are not unique, found duplicates for: {'RA5 ', 'RR10'}. Applying running numbers for duplicates.
  montage.plot();
<ipython-input-35-8c0958edec32>:3: RuntimeWarning: The following EEG sensors did not have a position specified in the selected montage: ['RA5 -1', 'RR10-1']. Their position has been left untouched.
  montage.plot();
/Users/shateeshbhugwansing/anaconda/envs/mne/lib/python3.6/site-packages/numpy/core/fromnumeric.py:83: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [39]:
# v4 is the file with the channels in the correct orientation
# had to manually transform the position array (x,y) --> (-y, x)
montage2 = mne.channels.read_montage(kind="ANT_DukeWaveGuard_128_electrode_montages_updated_V4")


In [40]:
from pylab import rcParams
rcParams['figure.figsize'] = 15, 15

montage2.plot(); 

In [ ]:
epoch.set_montage(montage)

In [ ]:
epoch.plot_sensors();

In [ ]:
epoch.plot_sensors('3d');

In [ ]:
## testing what the other montage file looks like (the example that Dave sent)

In [ ]:
montage2 = mne.channels.read_montage(kind="eetrak124.elc", ch_names=None, path=path, unit='mm', transform=False)

In [ ]:
montage2.plot();

In [ ]:
## testing MNE, built in montage
montage3 = mne.channels.read_montage('standard_1020')


In [ ]:
montage3.plot();

## Sanity Check:
What does the montage look like as a normal scatter plot?

In [ ]:
plt.scatter(montage.pos[:,1], montage.pos[:,0])

In [ ]:
# try: manually editing the montage file columns so that the x axis is the first column, y axis is second
montage_rotated = mne.channels.read_montage(kind="ANT_DukeWaveGuard_128_electrode_montage_rotated", ch_names=None, path=path, unit='mm', transform=False)

In [ ]:
montage_rotated.plot();

In [ ]:
montage.plot();

In [ ]:
path2 = '/Users/shateeshbhugwansing/Desktop/seniorprojecteeg/Visualization'
montage = mne.channels.read_montage(kind='eetrak124', path=path2, unit='mm')

In [ ]:
montage.plot()

### THEN WHY THE HECK IS IT ROTATED USING MONTAGE.PLOT??

## Mapping coefficients to the channels
Map the coefficients of Logistic Regression to the channels of the ANT Waveguard Duke cap

In [ ]:
# %run ../preprocessing/Artifact_Removal/preprocecssing_helpers.ipynb
# %run ../preprocessing/StimCodes.ipynb
# %run ../Classification/ConcatEpochTrails.ipynb
# %run ../PCA/Emmanuil-PCA.ipynb

from mne.decoding import Vectorizer
from sklearn.preprocessing import MinMaxScaler, Normalizer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.svm import SVC  # noqa
from sklearn.model_selection import ShuffleSplit  # noqa

from mne.decoding import UnsupervisedSpatialFilter
from sklearn.decomposition import PCA

from sklearn.metrics import confusion_matrix
from mne.viz import tight_layout

import time

In [ ]:
new_stim_code_event_ids = {
     
    "b-f2-wrd":1,
    "b-f1-wrd":2,
    "b-tg-wrd":3,

    "b-f2-spk":4,
    "b-f1-spk":5,
    "b-tg-spk":6,

    "b-f2-pic":7,
    "b-f1-pic":8,
    "b-tg-pic":9,

    "b-f2-snd":10,
    "b-f1-snd":11,
    "b-tg-snd":12,

    "f-f2-wrd":13,
    "f-f1-wrd":14,
    "f-tg-wrd":15,

    "f-f2-spk":16,
    "f-f1-spk":17,
    "f-tg-spk":18,

    "f-f2-pic":19,
    "f-f1-pic":20,
    "f-tg-pic":21,

    "f-f2-snd":22,
    "f-f1-snd":23,
    "f-tg-snd":24
}

In [ ]:
## the following is from Emmanuil's Audio vs. Visual notebook

In [ ]:
# get indicies of trials of epoch object
trial_index_list = get_trial_index_list(epoch_object= epoch)

In [ ]:
# get new events for epoch object
new_event_list = convert_event_ids_to_stim_combinations(epoch_object=epoch,
                                                        trial_index_list = trial_index_list,
                                                        stim_combinations = stim_combinations)

In [ ]:
print(epoch.events.shape)
print(new_event_list.shape)

In [ ]:
# assign new events to current epoch object
epoch.events = new_event_list

# assign new event_ids to current epoch object (dictionary 
# found in ../Classification/ConcatEpochTrails.ipynb)
 
epoch.event_id = modality_lexicality_event_ids

In [ ]:
# classification time!

# Isolate audio vs visual codes
# audio codes are < 700, # visual codes are > 700 
# audio : 100 , visual : 101
for event in epoch.events:
    if event[-1] < 700:
        event[-1] = 100
    else:
        event[-1] = 101

In [ ]:
norm_pipe = make_pipeline(Vectorizer(), 
                          StandardScaler())
norm_pipe.fit(epoch.get_data())

In [ ]:
scaled_data = norm_pipe.transform(epoch.get_data())
scaled_data.shape

In [ ]:
# do PCA to reduce dimensions and data needed for classification.
# pca = UnsupervisedSpatialFilter(PCA(28), average=False) # PCA, keep 9 components 

epoch_data = epoch.get_data()
# pca_data = pca.fit_transform(epoch_data)

In [ ]:
# classification pipeline
start = time.time()
clf = make_pipeline(Vectorizer(),
                    StandardScaler(),
                    LogisticRegression(penalty='l1'))
end = time.time()
print("clf elapsed time: {0}".format(end - start))

start_master = time.time()
cv = StratifiedKFold(n_splits=5, shuffle = True, random_state = 42)

labels = epoch.events[:,-1]
preds = np.empty(len(epoch_data))
for train, test in cv.split(epoch_data, labels):
    start = time.time()
    clf.fit(epoch_data[train], labels[train])
    preds[test] = clf.predict(epoch_data[test])
    end = time.time()
    print("kfold elapsed time: {0}".format(end - start))
end = time.time()
print("classification elapsed time: {0}".format(end - start_master))


start = time.time()
target_names = ['Audio', 'Visual']
report = classification_report(labels, preds, target_names=target_names)
print(report)
end = time.time()
print("reporting elapsed time: {0}".format(end - start))

## Emmanuil's classification code crashed. 
## Going to use what Tarekul did.

In [ ]:
def update_stim_code(epoch):
    keys_to_delete = [x for x in epoch.event_id if 'stim' in x]
    for key in keys_to_delete:
        if 'stim' in key:
            del epoch.event_id[key]
            
    for key in new_stim_code_event_ids:
        epoch.event_id[key] = new_stim_code_event_ids[key]
    
    return epoch

In [ ]:
epochWnewStim = update_stim_code(epoch)
epochWnewStim.events.shape

In [ ]:
events = epochWnewStim.events[:,-1]
event_ids = epochWnewStim.event_id

In [ ]:
stim_combinations = {
    (5,6,4): "AALL",
    (17,18,16) :"AALL",
    
    (11,6,10):"AALN",
    (23,18,22):"AALN",
    
    (5,12,4) :"AANL",
    (17,24,16):"AANL",
    
    (11,12,10) : "AANN",
    (23,24,22): "AANN",
    
    (2,6,1):"AVLL",
    (14,18,13):"AVLL",
    
    (8,12,7):"AVNN",
    (20,24,19):"AVNN",
    
    (5,3,4) :"VALL",
    (17,15,16):"VALL",
    
    (11,9,10) :"VANN",
    (23,21,22):"VANN",
    
    (2,3,1) :"VVLL",
    (14,15,13):"VVLL",
    
    (8,3,7):"VVLN",
    (20,15,19):"VVLN",
    
    (2,9,1) :"VVNL",
    (14,21,13):"VVNL",
    
    (8,9,7) :"VVNN",
    (20,21,19):"VVNN",
    
    (14,21,4):"whatever"
    
    
    
}

In [ ]:
events = epochWnewStim.events
events_new = []
temp = []
combo = []


for event in events:
    if event[-1] == 31:
        combo = []
        for trialEpochs in temp:
            if trialEpochs[-1] >=1 and trialEpochs[-1]<=24:
                combo.append(trialEpochs[-1])
        if len(combo) == 3: 
            combo_tuple = (combo[0],combo[1],combo[2]) 
            if stim_combinations[combo_tuple] == "VVNN" :
                #print(combo_tuple)
                for k in temp:
                    if k[-1] >= 1 and k[-1] <= 24:
                        #print(k)
                        k[-1] = 100
            elif stim_combinations[combo_tuple] == "AANN":
                #print(combo_tuple)
                for k in temp:
                    if k[-1] >= 1 and k[-1] <= 24:
                        #print(k)
                        k[-1] = 200       
            combo_tuple = []
                
        
        for p in temp:
            if p[-1] == 100 or p[-1] == 200:
                events_new.append(p)
#             if p[-1] >=1 and p[-1] <=24:
#                 events_new.append(p)
        temp = []
        temp.append(event)
    else:
        temp.append(event)
events_new

In [ ]:
#New epoch with only visual stim code combinations
test = epochWnewStim[(epochWnewStim.events[:,-1]==100) | (epochWnewStim.events[:,-1]==200)].copy()

In [ ]:
from sklearn.cross_validation import train_test_split

#set features as epoch data
#set targets as event ids
features_data = test.get_data()
target_V_vs_A = test.events[:,-1]

print("shape of data",features_data.shape)
print("shape of target array",target_V_vs_A.shape)




In [ ]:
#new epoch with only events corresponding to 100 or 200(visual/audio stim codes)
binaryEpoch = epochWnewStim[(epochWnewStim.events[:,-1]==100) | (epochWnewStim.events[:,-1]==200)].copy()

In [ ]:
#pipeling for converting data from 3D to 2D using vectorizer and
#LogisticRegression for classification
clf = make_pipeline(Vectorizer(),
                    MinMaxScaler(),
                    (LogisticRegression(C=1)))

#Provides train/test indices to split data in train/test sets.
cv = StratifiedKFold(n_splits = 10, shuffle=True)
#y_preds = np.empty(len(target_V_vs_A))
score = []
# tprs = []
# aucs = []
# mean_fpr = np.linspace(0, 1, 100)

#i = 0
for train,test in cv.split(features_data,target_V_vs_A):
    X_train = features_data[train] 
    #print("shape of training features(data):",X_train.shape)
    y_train = target_V_vs_A[train] 
    #print("shape of training target:",y_train.shape)
    X_test = features_data[test] 
    #print("shape of testing data:",X_test.shape)
    y_test = target_V_vs_A[test]
    #print("shape of testing target:",y_test.shape)
    
    #fit the model to training set
    clf.fit(X_train,y_train)
    
    #returns the mean accuracy on test data and labels
    score.append(clf.score(X_test,y_test))
    
    
    




In [ ]:
print("mean classification score", np.mean(score))

In [ ]:
from mne.decoding import get_coef
#get the coeficients after fitting clf to training sets
patterns2 = get_coef(clf,'coef_')
#print(patterns2)

#convert patterns2 to 129 x 180 array from 1 x 23220
#take each block of 180 coeficients and put them together 
#for each channel. We will have 129 channels rows with
#180 time columns
hold = []
for h in range(0,23219,180):
    temp = []
    for i in range(h,h+180):
        temp.append(patterns2[0][i])
    hold.append(temp)     
npHold = np.array(hold)
npHold.shape

In [ ]:
len(epoch.ch_names)

In [ ]:
npHold_avg = []

for i in range(len(npHold)):
    npHold_avg.append(np.mean(npHold[i]))

# npHold_avg = np.array(npHold_avg)
print (len(npHold_avg))


In [ ]:
channels = epoch.ch_names


In [ ]:
channels_and_coeff = dict(zip(channels, npHold_avg))
channels_and_coeff

In [ ]:
# by observation, the averaged coefficients seem to be biggest on the scale of 0.001 

coeff_of_interest = {k:v for (k,v) in channels_and_coeff.items() if abs(v) > 0.001 }

In [ ]:
coeff_of_interest

In [ ]:
# Taking the average didn't work. Trying a new approach
# Interpolation of 2d data. 
# reference: https://docs.scipy.org/doc/scipy-0.14.0/reference/tutorial/interpolate.html#using-radial-basis-functions-for-smoothing-interpolation

# set up x,y coordinates by taking the min, max of x,y coordinates of montage file. 
x_min = np.amin(montage.pos[:,1])
x_max = np.amax(montage.pos[:,1])
print ("x range: ", x_max, x_min)

y_min = np.amin(montage.pos[:,0])
y_max = np.amax(montage.pos[:,0])
print("y range: ", y_min, y_max)

In [ ]:
xi = np.linspace(y_min, y_max)
yi = np.linspace(y_min, y_max)

In [ ]:
type(xi)

In [ ]:
XI, YI = np.meshgrid(yi, yi)

In [ ]:
np.array(XI).shape

In [ ]:
# Grid is set up. as for the data:
# x = x-axis location of sensor (montage.pos[1])
# y = y-axis location of sensor (montag.pos[0])
# z = coefficient used in linear regression

print( len(montage.pos[:,1]), len(montage.pos[:,0]), len(npHold_avg))

In [ ]:
len(epoch.ch_names)

In [ ]:
len(montage.ch_names)

#### NOTE: What I'm doing here is wrong. I should have filtered out the channels that are NOT the same in the epoch.ch_name and montage.ch_name

In [ ]:
averaged_coefficients = npHold_avg[:-2]

In [ ]:
len(averaged_coefficients)

In [ ]:
print("Mine: ", type(x[0]), len(x), type(y[0]), len(y), type(z[0]), len(z))

In [ ]:
type(npHold)

In [ ]:
# set up scattered data 
from scipy.interpolate import Rbf

x = montage.pos[:,1]
y = montage.pos[:,0]
z = np.array(averaged_coefficients)

In [ ]:
# Have to remove the duplicate x,y coordinates in order to get Rbf to work
# NEED TO PRESERVE ORDER (can't sort them!!)

def unique_unsorted(arr):
    _, idx = np.unique(arr, return_index=True)
    
    return arr[np.sort(idx)], idx

In [ ]:
x, x_idx = unique_unsorted(x)
y, y_idx  = unique_unsorted(y)

In [ ]:
# Filter z to get rid of the same indicies that the x array got rid of. 
z = z[np.sort(y_idx)]
print(len(z))

In [ ]:
type(x[0])

In [ ]:
rbf = Rbf(x,y,z, epsilon=2)

In [ ]:
ZI = rbf(XI, YI)

In [ ]:
# plot the result
n = plt.Normalize(-2., 2.)
plt.subplot(1, 1, 1)
plt.pcolor(XI, YI, ZI, cmap=cm.jet, alpha=0.2)
plt.scatter(x, y, 100, z, cmap=cm.jet)
plt.title('RBF interpolation - multiquadrics')
plt.xlim(-.1, .1)
plt.ylim(-.1, .1)
plt.colorbar()

### try the same thing, with a different z-axis value. 
#### try z = npHold[0] ?

In [ ]:
z2 = npHold[:,0]

In [ ]:
z2 = z2[np.sort(y_idx)]


In [ ]:
z2.shape

In [ ]:
rbf2 = Rbf(x,y,z2, epsilon=2)

In [ ]:
ZI2 = rbf2(XI, YI)

In [ ]:
# plot the result
n = plt.Normalize(-2., 2.)
plt.subplot(1, 1, 1)
plt.pcolor(XI, YI, ZI2, cmap=cm.jet, alpha=0.2)
plt.scatter(x, y, 100, z2, cmap=cm.jet)
plt.title('RBF interpolation - Column 0')
plt.xlim(-.1, .1)
plt.ylim(-.1, .1)
plt.colorbar()

### Do this for a few other columns, to show change over time

In [ ]:
z3 = npHold[:,30]
z4 = npHold[:,60]
z5 = npHold[:,120]

In [ ]:
z3 = z3[np.sort(y_idx)]
z4 = z4[np.sort(y_idx)]
z5 = z5[np.sort(y_idx)]

In [ ]:
rbf3 = Rbf(x,y,z3, epsilon=2)
rbf4 = Rbf(x,y,z4, epsilon=2)
rbf5 = Rbf(x,y,z5, epsilon=2)

In [ ]:
ZI3 = rbf3(XI, YI)
ZI4 = rbf4(XI, YI)
ZI5 = rbf5(XI, YI)

In [ ]:
# plot the result
n = plt.Normalize(-2., 2.)
plt.subplot(1, 1, 1)
plt.pcolor(XI, YI, ZI3, cmap=cm.jet, alpha=0.2)
plt.scatter(x, y, 100, z3, cmap=cm.jet)
plt.title('RBF interpolation - column 30')
plt.xlim(-.1, .1)
plt.ylim(-.1, .1)
plt.colorbar()

In [ ]:
# plot the result
n = plt.Normalize(-2., 2.)
plt.subplot(1, 1, 1)
plt.pcolor(XI, YI, ZI4, cmap=cm.jet, alpha=0.2)
plt.scatter(x, y, 100, z4, cmap=cm.jet)
plt.title('RBF interpolation - column 60')
plt.xlim(-.1, .1)
plt.ylim(-.1, .1)
plt.colorbar()

In [ ]:
# plot the result
n = plt.Normalize(-2., 2.)
plt.subplot(1, 1, 1)
plt.pcolor(XI, YI, ZI5, cmap=cm.jet, alpha=0.2)
plt.scatter(x, y, 100, z5, cmap=cm.jet)
plt.title('RBF interpolation - column 120')
plt.xlim(-.1, .1)
plt.ylim(-.1, .1)
plt.colorbar()

### Topography Map

In [ ]:
projs = mne.compute_proj_epochs(epoch)

In [ ]:
epoch_projs = epoch.add_proj(projs)

In [ ]:
type(epoch_projs)

In [ ]:
type(epoch)

In [ ]:
epoch_projs.plot_projs_topomap()

In [ ]:
epoch.plot_projs_topomap()

### topomap() is not working. Maybe I need to manaully create a layout